In [ ]:
!apt-get install -y portaudio19-dev python3-pyaudio
!pip install SpeechRecognition pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
python3-pyaudio is already the newest version (0.2.11-1.3ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
!pip uninstall -y langchain langchain-community langchain-google-genai google-generativeai -q
!pip install langchain==0.2.16 langchain-community==0.2.16 google-generativeai==0.7.2 chromadb sentence-transformers -q
!pip install --upgrade pip
!pip install sentence-transformers
!pip install transformers
!pip install numpy --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
diffusers 0.35.2 requires huggingface-hub>=0.34.0, but you have huggingface-hub 0.23.5 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
datasets 4.0.0 requires huggingface-hub>=0.24.0, but you have huggingface-hub 0.23.5 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", b

In [ ]:
!pip install -U pip
!pip uninstall -y transformers huggingface-hub sentence-transformers
!pip install transformers==4.44.2 huggingface-hub==0.23.5 sentence-transformers==2.6.1

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: huggingface-hub 0.23.5
Uninstalling huggingface-hub-0.23.5:
  Successfully uninstalled huggingface-hub-0.23.5
Found existing installation: sentence-transformers 2.6.1
Uninstalling sentence-transformers-2.6.1:
  Successfully uninstalled sentence-transformers-2.6.1
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.23.5-py3-none-any.whl.metadata (12 kB)
  Using cached sentence_transformers-2.6.1-py3-none-any.whl.metadata (11 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached huggingface_hub-0.23.5-py3-none-any.whl (402 kB)
Using cached sentence_transformers-2.6.1-py3-none-any.whl (163 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [sentence-transformers]
ERROR: pip's dependency resolver does not currently take into account all the packages that are ins

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

🧠 LLM-Powered SQL Generation: Uses Gemini 2.0 Flash to convert plain English questions into optimized SQL queries.

💾 SQLite Integration: Connects to olist_sqlite.db, which contains all e-commerce tables.

🗣️ Conversational Memory: Remembers the last few interactions, enabling follow-up questions like “show me those from last month.”

⚙️ Error Handling & Rate Limit Protection: Waits and retries when API limits are hit.

📜 Interactive CLI Mode: Supports commands —

history → show previous Q&A

clear → reset chat memory

quit → exit the session

In [ ]:
# ==================== MULTI-AGENT SYSTEM WITH VOICE & IMPROVEMENTS ====================

import os
import time
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import google.generativeai as genai
from langchain_community.utilities import SQLDatabase
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import re
from datetime import datetime
import gradio as gr
from typing import Tuple, Optional, Dict, Any
import io
from PIL import Image
import speech_recognition as sr
from pydub import AudioSegment
import tempfile

# ==================== CONFIG ====================
os.environ["GOOGLE_API_KEY"] = ""#ADD YOUR API KEY HERE 
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Rate limit tracking
api_calls_made = 0
MAX_API_CALLS = 180

# ==================== DATABASE ====================
sqlite_path = "/content/drive/MyDrive/archive/olist_sqlite.db"#ADD YOUR SQLITE PATH HERE 
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_path}")

# ==================== SCHEMA INFO ====================
SCHEMA_INFO = """
Tables and Columns:

1. olist_customers_dataset (99,441 rows)
   Columns: customer_id, customer_unique_id, customer_zip_code_prefix, customer_city, customer_state

2. olist_order_payments_dataset (103,886 rows)
   Columns: order_id, payment_sequential, payment_type, payment_installments, payment_value

3. olist_order_items_dataset (112,650 rows)
   Columns: order_id, order_item_id, product_id, seller_id, shipping_limit_date, price, freight_value

4. olist_order_reviews_dataset (99,224 rows)
   Columns: review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp

5. olist_sellers_dataset (3,095 rows)
   Columns: seller_id, seller_zip_code_prefix, seller_city, seller_state

6. product_category_name_translation (71 rows)
   Columns: product_category_name, product_category_name_english

7. olist_products_dataset (32,951 rows)
   Columns: product_id, product_category_name, product_name_lenght, product_description_lenght, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm

8. olist_orders_dataset (99,441 rows)
   Columns: order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date
"""

# ==================== CHROMADB SETUP ====================
chroma_path = "/content/drive/MyDrive/chroma_db_download"#ADD YOUR CHROMADB PATH HERE 
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

vectorstores = {}
collection_names = [
    "olist_order_reviews_dataset",
    "olist_products_dataset",
    "olist_order_items_dataset",
    "product_category_name_translation"
]

try:
    for coll_name in collection_names:
        try:
            vs = Chroma(
                persist_directory=chroma_path,
                embedding_function=embedding_model,
                collection_name=coll_name
            )
            test = vs.similarity_search("test", k=1)
            if test:
                vectorstores[coll_name] = vs
        except:
            pass
    vectorstores = vectorstores if vectorstores else None
except:
    vectorstores = None

# ==================== GEMINI MODEL ====================
model = genai.GenerativeModel('gemini-2.0-flash')

# ==================== ENHANCED MEMORY & CACHE ====================
conversation_history = []
result_cache = {
    'last_sql_result': None,
    'last_plot_path': None,
    'last_sql_query': None,
    'last_question': None,
    'last_product_ids': [],
    'last_order_ids': [],
    'last_category': None,
    'query_history': []  # Store last 5 queries
}

# Dataset info
DATASET_START_YEAR = 2016
DATASET_END_YEAR = 2018
DATASET_INFO = f"Brazilian Olist e-commerce dataset (2016-2018): 100k orders from multiple marketplaces"

# ==================== VOICE AGENT ====================
def transcribe_audio(audio_path: str) -> Tuple[str, str]:
    """Transcribe audio to text using speech recognition"""
    if audio_path is None:
        return "", "❌ No audio provided"

    try:
        recognizer = sr.Recognizer()

        # Handle different audio formats
        audio_file = sr.AudioFile(audio_path)

        with audio_file as source:
            # Adjust for ambient noise
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio_data = recognizer.record(source)

        # Try Google Speech Recognition
        try:
            text = recognizer.recognize_google(audio_data)
            return text, f"✅ **Transcribed:** '{text}'"
        except sr.UnknownValueError:
            return "", "❌ Could not understand audio. Please speak clearly."
        except sr.RequestError:
            # Fallback to Sphinx if Google API fails
            try:
                text = recognizer.recognize_sphinx(audio_data)
                return text, f"✅ **Transcribed (offline):** '{text}'"
            except:
                return "", "❌ Speech recognition service unavailable"

    except Exception as e:
        return "", f"❌ Audio processing error: {str(e)}"

# ==================== CONTEXT RESOLUTION ====================
def resolve_context(question: str) -> str:
    """Resolve references to previous queries"""
    question_lower = question.lower()

    # Check for contextual references
    context_keywords = ['that', 'this', 'previous', 'last', 'it', 'them', 'those']
    has_context = any(keyword in question_lower for keyword in context_keywords)

    if not has_context:
        return question

    # Handle "plot that/this/previous"
    if any(word in question_lower for word in ['plot', 'visualize', 'chart', 'graph']):
        if result_cache['last_sql_result'] is not None:
            return question  # Will be handled by PLOT agent

    # Handle "show reviews for that/previous"
    if 'review' in question_lower and has_context:
        if result_cache['last_category']:
            return f"show reviews for {result_cache['last_category']} products"
        elif result_cache['last_product_ids']:
            return question  # Will use cached product IDs

    # Replace contextual references with actual entities
    resolved = question
    if result_cache['last_category']:
        resolved = resolved.replace('that category', result_cache['last_category'])
        resolved = resolved.replace('this category', result_cache['last_category'])

    return resolved

# ==================== DATE VALIDATION ====================
def validate_date_range(question: str) -> Tuple[bool, Optional[str]]:
    """Check if question asks for data outside available range"""
    years = re.findall(r'\b(20\d{2})\b', question)

    if not years:
        return True, None

    invalid_years = []
    for year in years:
        year_int = int(year)
        if year_int < DATASET_START_YEAR or year_int > DATASET_END_YEAR:
            invalid_years.append(year)

    if invalid_years:
        error_msg = f"❌ **Data not available for year(s): {', '.join(invalid_years)}**\n\n"
        error_msg += f"📊 {DATASET_INFO}\n\n"
        error_msg += f"✅ **Available years:** {DATASET_START_YEAR}-{DATASET_END_YEAR}\n\n"
        error_msg += f"💡 **Try:** 'most sold product in 2017' or 'revenue in 2018'"
        return False, error_msg

    return True, None

# ==================== SAFE API CALL ====================
def safe_api_call(prompt, max_tokens=500, temperature=0):
    """Make API call with rate limit checking"""
    global api_calls_made

    if api_calls_made >= MAX_API_CALLS:
        return None

    try:
        response = model.generate_content(
            prompt,
            generation_config=genai.GenerationConfig(
                temperature=temperature,
                max_output_tokens=max_tokens
            )
        )
        api_calls_made += 1
        return response.text.strip()
    except Exception as e:
        if "429" in str(e) or "quota" in str(e).lower():
            return None
        return None

# ==================== RULE-BASED ROUTING ====================
def rule_based_routing(question: str) -> list:
    """Enhanced rule-based routing with context awareness"""
    question_lower = question.lower()

    # Check for context references
    if any(word in question_lower for word in ['previous', 'that', 'this', 'last']):
        if 'plot' in question_lower or 'visualize' in question_lower:
            return ['PLOT']
        if 'review' in question_lower:
            return ['RAG']

    if any(word in question_lower for word in ['plot', 'graph', 'chart', 'visualize', 'show']):
        return ['SQL', 'PLOT']

    if any(word in question_lower for word in ['what is', 'explain', 'define', 'meaning of']):
        return ['WEB', 'SQL', 'RAG']

    if any(word in question_lower for word in ['review', 'feedback', 'customer', 'opinion']):
        return ['SQL', 'RAG']

    return ['SQL', 'RAG']

def route_question(question: str) -> list:
    """Determines which agents to use"""
    routing_prompt = f"""Analyze and determine agents:

Question: {question}

Agents: SQL, RAG, PLOT, WEB
Return ONE LINE: "SQL,RAG" or "SQL,PLOT" etc.

Response:"""

    result = safe_api_call(routing_prompt, max_tokens=50)

    if result is None:
        return rule_based_routing(question)

    try:
        agents = result.upper().split(',')
        agents = [a.strip() for a in agents if a.strip() in ['SQL', 'RAG', 'PLOT', 'WEB']]
        return agents if agents else ['SQL', 'RAG']
    except:
        return rule_based_routing(question)

# ==================== SQL GENERATION ====================
def generate_sql_rule_based(question: str) -> str:
    """Generate SQL using patterns"""
    question_lower = question.lower()

    year_match = re.search(r'\b(20\d{2})\b', question)
    year = None
    if year_match:
        year_int = int(year_match.group(1))
        if DATASET_START_YEAR <= year_int <= DATASET_END_YEAR:
            year = year_match.group(1)

    # Pattern: Average order value / average price
    if 'average' in question_lower and ('order value' in question_lower or 'price' in question_lower):
        category_match = re.search(r'(?:in|for|of)\s+(?:the\s+)?(\w+(?:\s+\w+)?)\s+category', question_lower)

        if category_match:
            category = category_match.group(1).strip()
            where_clause = f"WHERE t.product_category_name_english LIKE '%{category}%'"
        else:
            where_clause = ""

        return f"""SELECT AVG(oi.price) as average_order_value
FROM olist_order_items_dataset oi
JOIN olist_products_dataset p ON oi.product_id = p.product_id
JOIN product_category_name_translation t ON p.product_category_name = t.product_category_name
{where_clause}"""

    if 'most sold' in question_lower or 'top sell' in question_lower or 'best sell' in question_lower:
        where_clause = f"WHERE strftime('%Y', o.order_purchase_timestamp) = '{year}'" if year else ""
        return f"""SELECT oi.product_id, COUNT(*) as sales_count, t.product_category_name_english, SUM(oi.price) as total_revenue
FROM olist_order_items_dataset oi
JOIN olist_orders_dataset o ON oi.order_id = o.order_id
JOIN olist_products_dataset p ON oi.product_id = p.product_id
JOIN product_category_name_translation t ON p.product_category_name = t.product_category_name
{where_clause}
GROUP BY oi.product_id, t.product_category_name_english
ORDER BY sales_count DESC
LIMIT 1"""

    if 'revenue' in question_lower and 'category' in question_lower:
        where_clause = f"WHERE strftime('%Y', o.order_purchase_timestamp) = '{year}'" if year else ""
        return f"""SELECT t.product_category_name_english, COUNT(*) as sales_count, SUM(oi.price) as total_revenue
FROM olist_order_items_dataset oi
JOIN olist_orders_dataset o ON oi.order_id = o.order_id
JOIN olist_products_dataset p ON oi.product_id = p.product_id
JOIN product_category_name_translation t ON p.product_category_name = t.product_category_name
{where_clause}
GROUP BY t.product_category_name_english
ORDER BY total_revenue DESC
LIMIT 20"""

    if 'seller' in question_lower and ('top' in question_lower or 'best' in question_lower):
        limit = 25 if '25' in question else 10
        return f"""SELECT seller_id, COUNT(*) as sales_frequency
FROM olist_order_items_dataset
GROUP BY seller_id
ORDER BY sales_frequency DESC
LIMIT {limit}"""

    if 'top' in question_lower and 'product' in question_lower:
        limit_match = re.search(r'\b(\d+)\b', question)
        limit = limit_match.group(1) if limit_match else 10
        return f"""SELECT oi.product_id, COUNT(*) as sales_count, t.product_category_name_english
FROM olist_order_items_dataset oi
JOIN olist_products_dataset p ON oi.product_id = p.product_id
JOIN product_category_name_translation t ON p.product_category_name = t.product_category_name
GROUP BY oi.product_id, t.product_category_name_english
ORDER BY sales_count DESC
LIMIT {limit}"""

    return "Error: Could not generate SQL"

def generate_sql_from_question(question: str) -> str:
    prompt = f"""Generate SQL:

{SCHEMA_INFO}

Question: {question}

Rules:
- Start with SELECT, use exact columns
- strftime('%Y', order_purchase_timestamp) for year
- For category filtering, use product_category_name_english after joining with product_category_name_translation
- Return complete query with all necessary JOINs

SQL:"""

    result = safe_api_call(prompt, max_tokens=600)

    if result is None:
        return generate_sql_rule_based(question)

    lines = result.split('\n')
    clean_lines = []
    found_select = False
    for line in lines:
        if 'SELECT' in line.upper() or found_select:
            found_select = True
            clean_lines.append(line)

    sql = '\n'.join(clean_lines).replace('```sql','').replace('```','').strip()
    if sql.endswith(';'):
        sql = sql[:-1]
    return sql if sql else generate_sql_rule_based(question)

# ==================== WEB SEARCH ====================
def web_search_agent(question: str) -> str:
    """Search web"""
    knowledge_base = {
        "bed bath table": """Bed, Bath & Table products include:
- Bedding: sheets, pillowcases, comforters, duvets
- Bath items: towels, bath mats, shower curtains
- Table linens: tablecloths, placemats, napkins
Common buyers: homeowners, newlyweds, renters""",

        "health beauty": """Health & Beauty products include:
- Personal care: skincare, haircare, body care
- Cosmetics: makeup, fragrances, nail care
- Health: vitamins, supplements, medical supplies
Popular with: all demographics, especially women 18-45"""
    }

    question_lower = question.lower()
    for key, answer in knowledge_base.items():
        if key in question_lower:
            return answer

    result = safe_api_call(f"Briefly explain: {question}", max_tokens=300)
    return result if result else "Web search unavailable (rate limit)"

# ==================== PLOTTING ====================
def plotting_agent(question: str, data: pd.DataFrame = None) -> Optional[str]:
    """Create visualizations with cache support"""

    if data is None:
        data = result_cache['last_sql_result']

    if data is None or len(data) == 0:
        return None

    df = data.head(50)
    cols = list(df.columns)

    plot_type = 'bar'
    if 'time' in question.lower() or 'trend' in question.lower():
        plot_type = 'line'
    elif 'pie' in question.lower() or 'proportion' in question.lower():
        plot_type = 'pie'

    y_col = None
    x_col = None

    for col in cols:
        if df[col].dtype in ['int64', 'float64']:
            if 'count' in col.lower() or 'frequency' in col.lower() or 'revenue' in col.lower():
                y_col = col
                break

    if y_col is None:
        for col in cols:
            if df[col].dtype in ['int64', 'float64']:
                y_col = col
                break

    for col in cols:
        if col != y_col and (df[col].dtype == 'object' or col.endswith('_id')):
            x_col = col
            break

    if x_col is None:
        x_col = cols[0]

    try:
        plt.figure(figsize=(14, 7))

        if plot_type == 'bar':
            x_data = df[x_col]
            y_data = df[y_col] if y_col else df.iloc[:, -1]

            if df[x_col].dtype == 'object':
                x_labels = [str(x)[:15] + '...' if len(str(x)) > 15 else str(x) for x in x_data]
            else:
                x_labels = x_data

            plt.bar(range(len(y_data)), y_data, color='steelblue', alpha=0.8)
            plt.xticks(range(len(y_data)), x_labels, rotation=45, ha='right')
            plt.xlabel(x_col)
            plt.ylabel(y_col if y_col else 'Value')

        elif plot_type == 'line':
            y_data = df[y_col] if y_col else df.iloc[:, -1]
            plt.plot(range(len(df)), y_data, marker='o', linewidth=2, markersize=6)
            plt.xlabel('Index')
            plt.ylabel(y_col if y_col else 'Value')

        elif plot_type == 'pie':
            labels = [str(x)[:20] for x in df[x_col].head(10)]
            values = df[y_col].head(10) if y_col else df.iloc[:10, -1]
            plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=90)
            plt.axis('equal')

        title = f"{y_col if y_col else 'Data'} by {x_col}"
        plt.title(title, fontsize=14, fontweight='bold')
        plt.tight_layout()

        plot_filename = f"/content/plot_{int(time.time())}.png"
        plt.savefig(plot_filename, dpi=150, bbox_inches='tight')
        plt.close()

        result_cache['last_plot_path'] = plot_filename
        return plot_filename

    except Exception as e:
        return None

# ==================== ENHANCED RAG SEARCH ====================
def search_reviews_by_category(category: str, k: int = 5) -> str:
    """Fuzzy search reviews by category name"""
    if not vectorstores:
        return ""

    try:
        # First, get products in this category
        conn = sqlite3.connect(sqlite_path)
        query = f"""
        SELECT DISTINCT oi.product_id
        FROM olist_order_items_dataset oi
        JOIN olist_products_dataset p ON oi.product_id = p.product_id
        JOIN product_category_name_translation t ON p.product_category_name = t.product_category_name
        WHERE t.product_category_name_english LIKE '%{category}%'
        LIMIT 20
        """
        df = pd.read_sql_query(query, conn)
        conn.close()

        if len(df) > 0:
            return search_reviews_by_product_ids(df['product_id'].tolist(), k=k)
    except:
        pass

    # Fallback: direct semantic search
    if "olist_order_reviews_dataset" in vectorstores:
        try:
            vs = vectorstores["olist_order_reviews_dataset"]
            docs = vs.similarity_search(f"{category} product review", k=k)
            results = []
            for i, doc in enumerate(docs, 1):
                score = doc.metadata.get('review_score', 'N/A')
                content = doc.page_content[:250].strip()
                results.append(f"**Review {i}** (Score: {score}/5)\n{content}...")
            return "\n\n".join(results)
        except:
            pass

    return ""

def search_reviews_by_order_ids(order_ids: list, k: int = 5) -> str:
    """Search reviews by order IDs"""
    if not vectorstores or "olist_order_reviews_dataset" not in vectorstores:
        return ""

    vs = vectorstores["olist_order_reviews_dataset"]
    all_docs = []

    for order_id in order_ids[:5]:
        try:
            docs = vs.similarity_search(str(order_id), k=2)
            for doc in docs:
                if doc.metadata.get('order_id') == order_id:
                    all_docs.append(doc)
        except:
            continue

    if not all_docs:
        try:
            all_docs = vs.similarity_search("customer review feedback", k=k)
        except:
            return ""

    results = []
    for i, doc in enumerate(all_docs[:k], 1):
        score = doc.metadata.get('review_score', 'N/A')
        content = doc.page_content[:250].strip()
        results.append(f"**Review {i}** (Score: {score}/5)\n{content}...")

    return "\n\n".join(results)

def search_reviews_by_product_ids(product_ids: list, k: int = 5) -> str:
    """Search reviews by product IDs"""
    if not vectorstores:
        return ""

    try:
        conn = sqlite3.connect(sqlite_path)
        product_ids_str = "','".join([str(pid) for pid in product_ids[:5]])
        query = f"""
        SELECT DISTINCT o.order_id
        FROM olist_order_items_dataset o
        WHERE o.product_id IN ('{product_ids_str}')
        LIMIT 15
        """
        df = pd.read_sql_query(query, conn)
        conn.close()

        if len(df) > 0:
            return search_reviews_by_order_ids(df['order_id'].tolist(), k=k)
    except:
        pass

    return ""

# ==================== UPDATE CACHE ====================
def update_result_cache(question: str, sql_result: pd.DataFrame, sql_query: str):
    """Update result cache with latest query info"""
    result_cache['last_sql_result'] = sql_result
    result_cache['last_sql_query'] = sql_query
    result_cache['last_question'] = question

    # Extract useful metadata
    if sql_result is not None and len(sql_result) > 0:
        if 'product_id' in sql_result.columns:
            result_cache['last_product_ids'] = sql_result['product_id'].tolist()[:20]

        if 'order_id' in sql_result.columns:
            result_cache['last_order_ids'] = sql_result['order_id'].tolist()[:20]

        if 'product_category_name_english' in sql_result.columns:
            result_cache['last_category'] = sql_result['product_category_name_english'].iloc[0]

    # Store in query history (keep last 5)
    result_cache['query_history'].append({
        'question': question,
        'timestamp': datetime.now().strftime('%H:%M:%S'),
        'has_result': sql_result is not None
    })
    if len(result_cache['query_history']) > 5:
        result_cache['query_history'].pop(0)

# ==================== MAIN PROCESSING FUNCTION ====================
def process_question(question: str, history: list, audio_input=None) -> Tuple[str, Optional[str], str, str]:
    """Process question (text or voice) and return answer, plot, SQL, and transcription"""

    transcription_msg = ""

    # Handle voice input
    if audio_input is not None:
        transcribed_text, trans_msg = transcribe_audio(audio_input)
        transcription_msg = trans_msg
        if transcribed_text:
            question = transcribed_text
        else:
            return trans_msg, None, "", trans_msg

    if not question.strip():
        return "Please enter a question or record audio.", None, "", transcription_msg

    # Resolve contextual references
    original_question = question
    question = resolve_context(question)

    if original_question != question:
        transcription_msg += f"\n\n🔄 **Resolved to:** {question}"

    # Validate date range
    is_valid, error_msg = validate_date_range(question)
    if not is_valid:
        return error_msg, None, "", transcription_msg

    # Route to agents
    agents = route_question(question)

    status = f"🤖 **Agents:** {' → '.join(agents)}\n\n"

    web_result = ""
    sql_result = ""
    rag_results = ""
    plot_path = None
    sql_query = ""
    df = None

    # Execute agents
    for agent in agents:
        if agent == 'WEB':
            web_result = web_search_agent(question)
            status += f"🌐 **Web Search:** Completed\n"

        elif agent == 'SQL':
            status += f"📊 **SQL Agent:** Generating query...\n"
            sql = generate_sql_from_question(question)

            if sql.startswith("Error"):
                status += f"❌ {sql}\n"
                continue

            sql_query = sql
            status += f"⚙️ **SQL Agent:** Executing query...\n"

            try:
                conn = sqlite3.connect(sqlite_path)
                df = pd.read_sql_query(sql, conn)
                conn.close()

                if len(df) == 0:
                    status += f"⚠️ No results found\n"
                    continue

                # Update cache
                update_result_cache(question, df, sql_query)

                # Format SQL result
                if len(df.columns) == 1 and len(df) == 1:
                    col_name = df.columns[0]
                    value = df.iloc[0, 0]
                    if pd.notna(value):
                        if isinstance(value, float):
                            sql_result = f"**{col_name}:** ${value:.2f}" if 'price' in col_name.lower() or 'revenue' in col_name.lower() else f"**{col_name}:** {value:.2f}"
                        else:
                            sql_result = f"**{col_name}:** {value}"
                    else:
                        sql_result = f"**{col_name}:** No data available"
                else:
                    sql_result = df.head(20).to_markdown(index=False)

                status += f"✅ **SQL Agent:** Found {len(df)} rows\n"

            except Exception as e:
                status += f"❌ **SQL Error:** {str(e)}\n"
                continue

        elif agent == 'RAG':
            status += f"🔍 **RAG Agent:** Searching reviews...\n"

            # Use cached data if available
            if result_cache['last_product_ids']:
                rag_results = search_reviews_by_product_ids(result_cache['last_product_ids'], k=5)
            elif result_cache['last_order_ids']:
                rag_results = search_reviews_by_order_ids(result_cache['last_order_ids'], k=5)
            elif result_cache['last_category']:
                rag_results = search_reviews_by_category(result_cache['last_category'], k=5)
            elif df is not None and len(df) > 0:
                if 'product_id' in df.columns:
                    product_ids = df['product_id'].tolist()
                    rag_results = search_reviews_by_product_ids(product_ids, k=5)
                elif 'order_id' in df.columns:
                    order_ids = df['order_id'].tolist()
                    rag_results = search_reviews_by_order_ids(order_ids, k=5)

            if rag_results:
                status += f"✅ **RAG Agent:** Found reviews\n"
            else:
                status += f"⚠️ **RAG Agent:** No reviews found\n"

        elif agent == 'PLOT':
            status += f"📈 **Plot Agent:** Creating visualization...\n"
            plot_path = plotting_agent(question)
            if plot_path:
                status += f"✅ **Plot Agent:** Chart created\n"
            else:
                status += f"⚠️ **Plot Agent:** Could not create plot\n"

    # Generate answer
    if not any([web_result, sql_result, rag_results]):
        return status + "\n❌ No results from any agent", None, sql_query, transcription_msg

    answer_prompt = f"""Synthesize this data:

Question: {question}
{'Web: ' + web_result[:500] if web_result else ''}
{'SQL: ' + sql_result[:1000] if sql_result else ''}
{'Reviews: ' + rag_results[:500] if rag_results else ''}

Provide clear insights:"""

    answer = safe_api_call(answer_prompt, max_tokens=500, temperature=0.3)

    if answer is None:
        answer = "**Results Summary:**\n\n"
        if sql_result:
            answer += f"📊 **Query Result:**\n\n{sql_result}\n\n"
        if rag_results:
            answer += f"📄 **Customer Reviews:**\n{rag_results[:500]}\n\n"
        if web_result:
            answer += f"🌐 **Web Info:**\n{web_result[:300]}\n\n"
        answer += f"\n\n_(API rate limit reached. Showing raw results. Calls: {api_calls_made}/{MAX_API_CALLS})_"

    # Build final output
    final_output = status + "\n---\n\n## 📝 Answer\n\n" + answer

    if sql_result and not sql_result.startswith("**"):
        final_output += f"\n\n---\n\n## 📊 Data Preview\n\n{sql_result}"

    if rag_results:
        final_output += f"\n\n---\n\n## 💬 Customer Reviews\n\n{rag_results[:500]}"

    return final_output, plot_path, sql_query, transcription_msg

# ==================== GRADIO INTERFACE ====================
def create_ui():
    with gr.Blocks(theme=gr.themes.Soft(), title="Multi-Agent E-commerce Analytics") as demo:
        gr.Markdown("""
        # 🚀 Multi-Agent E-commerce Analytics System
        ### Brazilian Olist Dataset (2016-2018) | 100k+ Orders | 🎤 Voice Enabled

        **Available Agents:** SQL 📊 | RAG 📄 | PLOT 📈 | WEB 🌐 | VOICE 🎤
        """)

        with gr.Row():
            with gr.Column(scale=2):
                # Voice input
                with gr.Group():
                    gr.Markdown("### 🎤 Voice Input (Optional)")
                    audio_input = gr.Audio(
                        sources=["microphone"],
                        type="filepath",
                        label="Record Your Question",
                        show_label=False
                    )
                    transcription_output = gr.Markdown(value="", label="Transcription")

                # Text input
                with gr.Group():
                    gr.Markdown("### 💬 Text Input")
                    question_input = gr.Textbox(
                        label="Ask a Question",
                        placeholder="e.g., 'Top 5 selling products in 2017' or 'plot that' or 'show reviews for previous'",
                        lines=2
                    )

                with gr.Row():
                    submit_btn = gr.Button("🔍 Analyze", variant="primary", scale=2)
                    clear_btn = gr.Button("🗑️ Clear", scale=1)

                gr.Markdown("""
                ### 💡 Example Queries:
                **Basic Queries:**
                - `Top selling product in 2017`
                - `Revenue by category in 2018 and plot it`
                - `Average order value for electronics category`

                **Contextual Queries (using cache):**
                - `plot that` _(plots previous SQL result)_
                - `show reviews for that` _(shows reviews for previous category)_
                - `visualize the previous data`

                **Category Queries:**
                - `What is bed bath table category?`
                - `Customer reviews for health_beauty products`
                - `Top 25 sellers and visualize`

                **🎤 Voice Commands:**
                - Click microphone and speak naturally
                - Example: "Show me top products in twenty seventeen"
                """)

            with gr.Column(scale=1):
                api_status = gr.Markdown(f"**API Usage:** 0/{MAX_API_CALLS}")
                dataset_info = gr.Markdown(f"**Dataset:** {DATASET_INFO}")

                with gr.Accordion("📋 Query History", open=False):
                    history_display = gr.Markdown("No queries yet")

                cache_status = gr.Markdown("**Cache:** Empty")

        with gr.Row():
            with gr.Column(scale=3):
                answer_output = gr.Markdown(label="Analysis Results", value="")

            with gr.Column(scale=2):
                plot_output = gr.Image(label="Visualization", type="filepath")

        with gr.Accordion("🔧 SQL Query (Generated)", open=False):
            sql_output = gr.Code(label="SQL Query", language="sql")

        with gr.Accordion("📚 Help & Information", open=False):
            gr.Markdown("""
            ### 🤖 Agent Capabilities:

            **VOICE Agent 🎤** _(NEW!)_
            - Speech-to-text transcription
            - Supports natural voice commands
            - Works with Google Speech Recognition
            - Fallback to offline recognition

            **SQL Agent 📊**
            - Query database for sales, products, orders, customers
            - Automatically generates SQL from natural language
            - Supports date filtering (2016-2018)
            - **Cache support:** Results stored for follow-up queries

            **RAG Agent 📄** _(ENHANCED!)_
            - Search customer reviews
            - Find product feedback by category
            - Fuzzy search by keywords
            - Uses cached product/order IDs for context

            **Plot Agent 📈** _(IMPROVED!)_
            - Create bar charts, line graphs, pie charts
            - Automatic visualization type selection
            - **Works with cached results** - say "plot that"
            - No need to re-run SQL queries

            **Web Agent 🌐**
            - Explain product categories
            - Provide general knowledge
            - Define terms

            ### 🆕 New Features:

            **1. Voice Input**
            - Click microphone icon to record
            - Speak your question naturally
            - Automatic transcription to text
            - Works with all agent types

            **2. Enhanced Caching**
            - Previous SQL results stored automatically
            - Say "plot that" to visualize last query
            - Say "show reviews for previous" for context
            - Maintains last 5 query history

            **3. Better Review Mapping**
            - Fuzzy search by category name
            - Keyword-based review search
            - No need for exact product IDs

            **4. Context Resolution**
            - Understands "that", "previous", "last"
            - Automatically resolves references
            - Seamless follow-up questions

            ### ⚠️ Important Notes:
            - Dataset covers 2016-2018 only
            - Queries outside this range will be rejected
            - API rate limit: 180 calls per session
            - Voice recognition requires microphone access
            - Cache persists during session only
            """)

        def update_status():
            status = f"**API Usage:** {api_calls_made}/{MAX_API_CALLS}"

            # Update history
            history_text = "**Recent Queries:**\n\n"
            if result_cache['query_history']:
                for i, q in enumerate(reversed(result_cache['query_history']), 1):
                    emoji = "✅" if q['has_result'] else "❌"
                    history_text += f"{i}. {emoji} [{q['timestamp']}] {q['question'][:50]}...\n"
            else:
                history_text = "No queries yet"

            # Update cache status - FIXED: Check for None and empty DataFrame properly
            cache_text = "**Cache Status:**\n\n"
            has_cache_items = False

            if result_cache['last_sql_result'] is not None and not result_cache['last_sql_result'].empty:
                cache_text += f"✅ SQL Result: {len(result_cache['last_sql_result'])} rows\n"
                has_cache_items = True

            if result_cache['last_plot_path']:
                cache_text += f"✅ Plot: Available\n"
                has_cache_items = True

            if result_cache['last_category']:
                cache_text += f"✅ Category: {result_cache['last_category']}\n"
                has_cache_items = True

            if result_cache['last_product_ids']:
                cache_text += f"✅ Products: {len(result_cache['last_product_ids'])} IDs\n"
                has_cache_items = True

            if not has_cache_items:
                cache_text += "Empty"

            return status, history_text, cache_text

        def submit_question(question, audio):
            answer, plot, sql, transcription = process_question(question, [], audio)
            status, history, cache = update_status()
            return answer, plot, sql, status, history, cache, transcription

        def clear_all():
            global result_cache
            result_cache = {
                'last_sql_result': None,
                'last_plot_path': None,
                'last_sql_query': None,
                'last_question': None,
                'last_product_ids': [],
                'last_order_ids': [],
                'last_category': None,
                'query_history': []
            }
            status, history, cache = update_status()
            return "", None, "", status, history, cache, ""

        submit_btn.click(
            fn=submit_question,
            inputs=[question_input, audio_input],
            outputs=[answer_output, plot_output, sql_output, api_status,
                    history_display, cache_status, transcription_output]
        )

        question_input.submit(
            fn=submit_question,
            inputs=[question_input, audio_input],
            outputs=[answer_output, plot_output, sql_output, api_status,
                    history_display, cache_status, transcription_output]
        )

        clear_btn.click(
            fn=clear_all,
            inputs=[],
            outputs=[answer_output, plot_output, sql_output, api_status,
                    history_display, cache_status, transcription_output]
        )

    return demo

# ==================== LAUNCH ====================
if __name__ == "__main__":
    print("🚀 Initializing Enhanced Multi-Agent System...")
    print(f"✅ Database: Connected")
    print(f"✅ ChromaDB: {len(vectorstores) if vectorstores else 0} collections")
    print(f"✅ Gemini API: Ready")
    print(f"✅ Voice Recognition: Ready")
    print(f"📅 Dataset: {DATASET_INFO}")
    print(f"🆕 New Features: Voice Input, Enhanced Caching, Context Resolution")
    print("\n🌐 Launching Gradio UI...")

    demo = create_ui()
    demo.launch(
        share=True,
        debug=True,
        server_port=7860
    )

/tmp/ipython-input-4170683697.py:70: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/tmp/ipython-input-41

🚀 Initializing Enhanced Multi-Agent System...
✅ Database: Connected
✅ ChromaDB: 4 collections
✅ Gemini API: Ready
✅ Voice Recognition: Ready
📅 Dataset: Brazilian Olist e-commerce dataset (2016-2018): 100k orders from multiple marketplaces
🆕 New Features: Voice Input, Enhanced Caching, Context Resolution

🌐 Launching Gradio UI...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://10671f7355fd5aa895.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://10671f7355fd5aa895.gradio.live
